```{contents}
```
## Chat Model

In **LangChain**, a **Chat Model** is:

> A message-based LLM interface that accepts structured conversation turns (roles), supports tool/function calling, and enables agentic and RAG workflows.

It is **not just “chat”** — it is the **core execution engine** for modern LLM systems.

---

### Why Chat Models Exist

Earlier LLM APIs:

```
"text prompt" → "text completion"
```

Modern LLM systems require:

* Role separation
* Tool calling
* Multi-turn context
* Structured outputs
* Streaming

Chat Models were introduced to support this.

---

### Chat Model Interface (Key Difference)

#### Input

```python
List[BaseMessage]
```

#### Output

```python
AIMessage(
  content="answer",
  tool_calls=[...]
)
```

---

### Message Types (Very Important)

LangChain defines explicit message objects:

| Message Type  | Purpose               |
| ------------- | --------------------- |
| SystemMessage | Global instructions   |
| HumanMessage  | User input            |
| AIMessage     | Model response        |
| ToolMessage   | Tool execution result |

This is what enables **control and safety**.

---

### Basic Chat Model Usage



In [1]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2
)

response = chat.invoke("Explain LangChain")
print(response.content)


LangChain is a framework designed to facilitate the development of applications that utilize large language models (LLMs). It provides a structured way to build applications that can leverage the capabilities of LLMs for various tasks, such as natural language understanding, text generation, and conversational agents. Here are some key components and features of LangChain:

1. **Modular Design**: LangChain is built with a modular architecture, allowing developers to easily integrate different components such as language models, data sources, and tools. This modularity enables flexibility in designing applications tailored to specific use cases.

2. **Chains**: One of the core concepts in LangChain is the idea of "chains." A chain is a sequence of operations that can involve multiple steps, such as retrieving data, processing it with an LLM, and then returning a response. Chains can be simple or complex, depending on the application's requirements.

3. **Agents**: LangChain supports the


Internally, LangChain converts the string into a `HumanMessage`.

---

### Explicit Message Control


In [ ]:

from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="You are an IT support assistant"),
    HumanMessage(content="VPN is not working")
]

response = chat.invoke(messages)
print(response.content)




This allows:

* Role-based behavior
* Priority ordering
* Instruction hierarchy

---

### Tool Calling (Chat Models Only)

Chat Models can **decide to call tools**.



In [ ]:

def ticket_count(source: str) -> int:
    return 120

chat_with_tools = chat.bind_tools([ticket_count])

response = chat_with_tools.invoke(
    "How many tickets are there in Jira?"
)

print(response.tool_calls)




What happens:

1. Model emits a tool call
2. LangChain parses it
3. Executor runs the tool
4. Tool result is fed back

---

### Structured Output (JSON Enforcement)



In [ ]:

from pydantic import BaseModel

class Ticket(BaseModel):
    category: str
    priority: str

structured_chat = chat.with_structured_output(Ticket)

result = structured_chat.invoke("CEO cannot access email")
print(result)




Benefits:

* No hallucinated formats
* Type-safe output
* Retry on validation failure

---

## 9. Streaming Tokens



In [ ]:
for chunk in chat.stream("Explain Chat Models"):
    print(chunk.content, end="")




Used for:

* Chat UIs
* SSE
* WebSockets

---

### Chat Models in RAG

```python
qa = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=retriever
)
```

Chat Model role:

* Read retrieved documents
* Synthesize answer
* Respect system instructions

---

## 11. Chat Models in Agents

Agents **require** Chat Models.

```python
agent = create_openai_tools_agent(
    llm=chat,
    tools=[ticket_count]
)
```

LLMs (text completion) **cannot do this**.

---

### Lifecycle of a Chat Model Call

```
Messages
 → Prompt assembly
 → Tool schema injection
 → Model invocation
 → Tool call parsing
 → Output parsing
 → Callbacks / tracing
```

---

### Why LangChain Uses Chat Models Everywhere

| Requirement       | Chat Model |
| ----------------- | ---------- |
| Multi-turn chat   | ✅          |
| Tool calling      | ✅          |
| Agents            | ✅          |
| Structured output | ✅          |
| Streaming         | ✅          |
| Safety & control  | ✅          |

---

### When NOT to Use Chat Models

Rare cases only:

* Legacy completion-only models
* Static batch text generation
* Migration scenarios

---

### Chat Model Providers

LangChain supports:

* OpenAI (`ChatOpenAI`)
* Anthropic (`ChatAnthropic`)
* Ollama (`ChatOllama`)
* Hugging Face (`ChatHuggingFace`)
* Azure OpenAI

Same interface, different backends.

---

### Common Mistakes

| Mistake                          | Result                |
| -------------------------------- | --------------------- |
| Using `LLM` instead of ChatModel | Limited functionality |
| Passing raw strings everywhere   | Loss of role control  |
| Ignoring system messages         | Unstable behavior     |
| Not using tool calling           | Manual routing logic  |

---

**Interview-Ready Summary**

> “Chat Models in LangChain are message-based, tool-aware LLM interfaces that enable agents, RAG, structured outputs, and streaming. They are the primary abstraction for building production-grade LLM systems.”